In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import pandas as pd
from neuralprophet import NeuralProphet
import plotly.graph_objects as go
import pickle

In [20]:
df = pd.read_csv('./dataset/sensors_202205251324.csv')

df['created_at'] = pd.to_datetime(df['created_at'], unit='s')
df.set_index('created_at',inplace=True)

cols = ['temperature']
df = df[cols].astype(float)


In [ ]:
df.head()

In [22]:
freq = '1min'

In [23]:
df = df.resample(freq).mean().interpolate()

In [ ]:
df.info()

In [25]:
df = df.reset_index()

In [26]:
new_column = df[['created_at', 'temperature']] 
new_column.dropna(inplace=True)
new_column.columns = ['ds', 'y'] 

In [ ]:
df_train, df_val = NeuralProphet().split_df(new_column, freq='1min', valid_p=0.2)

In [ ]:
n = NeuralProphet(
    changepoints_range=0.95,
    n_changepoints=120,
    batch_size=32,
    learning_rate=0.2,
    weekly_seasonality=False,
    daily_seasonality=True
)
# n = NeuralProphet()
model = n.fit(df_train, validation_df=df_val, freq='1h')

In [ ]:
trace1 = go.Scatter(
    x = model.index,
    y = model['SmoothL1Loss'].values,
    mode = 'lines',
    name = 'SmoothL1Loss'
)
trace2 = go.Scatter(
    x = model.index,
    y = model['SmoothL1Loss_val'].values,
    mode = 'lines',
    name = 'SmoothL1Loss_val'
)
trace3 = go.Scatter(
    x = model.index,
    y = model['MAE'].values,
    mode = 'lines',
    name = 'MAE'
)
trace4 = go.Scatter(
    x = model.index,
    y = model['MAE_val'].values,
    mode = 'lines',
    name = 'MAE_val'
)

layout = go.Layout(
    title = "temp",
    xaxis = {'title' : "datetime"},
    yaxis = {'title' : "temperature"}
)
fig = go.Figure(data=[trace1, trace2,trace3,trace4], layout=layout)
fig.show()

In [ ]:
future = n.make_future_dataframe(df_val, periods=24,n_historic_predictions=True)
forecast = n.predict(future)

In [ ]:
fig_forecast = n.plot(forecast)
fig_components = n.plot_components(forecast)

In [ ]:
trace1 = go.Scatter(
    x = new_column['ds'].values,
    y = new_column['y'].values,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast['ds'].values,
    y = forecast['yhat1'].values,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = "temp",
    xaxis = {'title' : "datetime"},
    yaxis = {'title' : "temperature"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [49]:
with open('./models/neuralprophet_model.pkl', "wb") as f:
    pickle.dump(n, f)